## При создании окна ему можно передать следующие параметры:

* **PARTITION BY** [city]
* **ORDER BY** [date ASC]
* **ROWS** [BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW]

– **UNBOUNDED PRECEDING** – указывает, что окно начинается с первой строки секции

– **UNBOUNDED FOLLOWING** – указывает, что окно заканчивается на последней строке секции

## PARTITION BY
Группирует строки запроса в разделы, которые потом обрабатываются оконной функцией независимо друг от друга. Работает аналогично блоку **GROUP BY** в запросе, только группирует данные для окна.

## ORDER BY
Сортирует результаты внутри раздела, тем самым определяя порядок, в котором оконная функция будет работать со строками. Работает аналогично блоку **ORDER BY** на уровне запроса.

## ROWS/RANGE
Задает параметры рамки окна. Рамки используются в тех оконных функциях, которые работают с рамками, а не с разделом целиком. Первым аргументом задается начало рамки, вторым аргументом конец рамки, и дополнительно, третьим аргументом задается исключение из рамки.

Рамку можно задать в нескольких режимах, а именно:

* **ROWS** – Начало и конец рамки задаются в строках относительно текущей строки. Например ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING означает создание рамки на 3 строки вверх и вниз относительно текущей строки.
* **RANGE** – Начало и конец рамки задаются в разнице значений столбца из ORDER BY. Например, если в ORDER BY находится столбец event_date с типом данных date, то определение окна можно задать следующим образом **RANGE BETWEEN '3 day' PRECEDING AND '3 days' FOLLOWING** что будет означать рамку на 3 дня назад и вперед.

При указании рамки через **RANGE** обязательным условием является только один столбец в **ORDER BY** окна.

----------------
Query example: 

```sql
SELECT
    sq.installationdate AS installationdate,    
    sq.platform AS platform,    
    sq.cost AS cost,    
    SUM(sq.cost) OVER w AS total_cost    
FROM
    (SELECT    
        installationdate,        
        platform,       
        SUM(installcost) AS cost        
    FROM installs    
    WHERE installcost > 0    
    GROUP BY installationdate,    
             platform             
    ) AS sq     
WINDOW w AS (
    PARTITION BY sq.platform    
    ORDER BY CAST(sq.installationdate AS date) ASC    
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW    
    )    
ORDER BY installationdate ASC,
         platform ASC
```

----------------

Query example: 
```sql
SELECT 
    sq.installationdate AS installationdate,    
    sq.cost AS cost,    
    SUM(sq.cost) OVER w AS total_cost,    
    AVG(sq.cost) OVER w AS avg_cost,    
    row_number() OVER w AS row_number,    
    LAG(sq.cost, 1) OVER w AS lag_test,    
    LEAD(sq.cost, 1) OVER w AS lead_test    
FROM
    (SELECT   
        installationdate,        
        SUM(installcost) AS cost        
    FROM installs    
    WHERE installcost > 0    
    GROUP BY installationdate    
    ) AS sq     
WINDOW w AS ()
ORDER BY installationdate ASC
```

------------

Query без объявления **WINDOW**:
```sql
SELECT 
    sq.installationdate AS installationdate,   
    sq.cost AS cost,    
    SUM(sq.cost) OVER () AS total_cost,    
    AVG(sq.cost) OVER () AS avg_cost,    
    row_number() OVER () AS row_number,    
    LAG(sq.cost, 1) OVER () AS lag_test,    
    LEAD(sq.cost, 1) OVER () AS lead_test    
FROM
    (SELECT    
        installationdate,       
        SUM(installcost) AS cost      
    FROM installs    
    WHERE installcost > 0    
    GROUP BY installationdate    
    ) AS sq   
ORDER BY installationdate ASC
```

---------------
[Документация PostgreSQL](https://postgrespro.ru/docs/postgresql)
